# Using MLFlow and Evidently to Evaluate Data Drift

In this example, we will explore the MLflow integration with Evidently.

This notebook shows how you can use the Evidently and MLflow to:
* calculate data drift for the model, performed as batch checks 
* log data drift using MLflow Tracking
* explore the result using MLflow UI


In [32]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import json
import pandas as pd
import plotly.graph_objects as go
import requests
import zipfile
import io

from datetime import datetime
from typing import List, Dict, Tuple, Union, Optional

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

More information about the dataset can be found in Kaggle Playground Competition: https://www.kaggle.com/c/bike-sharing-demand/data?select=train.csv

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [34]:
#load data
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("day.csv"), header=0, sep=',', parse_dates=['dteday'])

In [35]:
#observe data structure
raw_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [36]:
#set column mapping for Evidently Profile
data_columns = ColumnMapping()
data_columns.datetime = 'dteday'
data_columns.numerical_features = ['weathersit', 'temp', 'atemp', 'hum', 'windspeed']
data_columns.categorical_features = ['holiday', 'workingday']
data_columns.target = 'cnt'

## Dataset Drift

In [37]:

# # Evaluate data drift with Evidently Profile
# def get_data_drift_report(reference, production, column_mapping) -> Report:
#     """
#     Returns a list with pairs (feature_name, drift_score)
#     Drift Score depends on the selected statistical test or distance and the threshold
#     """    
#     data_drift_report = Report(metrics=[
#         DataDriftPreset(drift_share=0.4)
#         ])
#     data_drift_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)

#     return data_drift_report

# def get_data_drift_metrics(report: Dict) -> Dict:
#     """
#     Returns a list with pairs (feature_name, drift_score)
#     Drift Score depends on the selected statistical test or distance and the threshold
#     """ 
#     metrics = {}   
#     for metric in ['dataset_drift', 
#                    'number_of_drifted_columns', 
#                    'share_of_drifted_columns']: 
#         metrics.update({metric: report["metrics"][0]["result"][metric]})

#     return metrics

## Setup Experiments

- option 1: each period (fold) is a separate experiment Run -> save metrics for each Run
- option 2: all folds are part of a single expeirment -> aggrregate metrics
- 

In [38]:
# Set reference dates
reference_dates = ('2011-01-01 00:00:00','2011-01-28 23:00:00')

# Set experiment batches dates
experiment_batches = [
    ('2011-01-01 00:00:00','2011-01-29 23:00:00'),
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

## Setup MLFLow Experiment

In [39]:
from config import EXPERIMENT_NAME, MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

EXPERIMENT_NAME = "Data Drift"

# Get or Create an experiment by name 
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

if experiment: 
    
    experiment_id = experiment.experiment_id if experiment else None
    # Show experiment info
    print("Name: {}".format(experiment.name))
    print("Experiment ID: {}".format(experiment.experiment_id))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

else:
    
    # Create_experiment
    experiment_id = client.create_experiment(
        EXPERIMENT_NAME ,
        # artifact_location=MLFLOW_TRACKING_URI,
        # tags={"version": "v1", "priority": "P1"},
    )
    print("Experiment ID: {}".format(experiment_id))

Client tracking uri: http://localhost:5000
Name: Data Drift
Experiment ID: 234193429068143937
Artifact Location: mlflow-artifacts:/234193429068143937
Lifecycle_stage: active


### Option 1 

In [40]:
#start new run
for date in experiment_batches:
    with mlflow.start_run(experiment_id=experiment_id) as run: 
        
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Log metrics
        report: Dict = get_data_drift_report(
            raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
            raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
            column_mapping=data_columns)
        
        metrics: Dict = get_data_drift_metrics(report.as_dict())
        
        for metric in metrics.keys():
            mlflow.log_metric(metric, round(metrics[metric], 3))

        print(run.info)

cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
<RunInfo: artifact_uri='mlflow-artifacts:/234193429068143937/2deb7d7588b34338aabff4ff4b8433d1/artifacts', end_time=None, experiment_id='234193429068143937', lifecycle_stage='active', run_id='2deb7d7588b34338aabff4ff4b8433d1', run_name='vaunted-horse-105', run_uuid='2deb7d7588b34338aabff4ff4b8433d1', start_time=1688639921315, status='RUNNING', user_id='mnrozhkov'>
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
<RunInfo: artifact_uri='mlflow-artifacts:/234193429068143937/769e310f88e84dad8806598a23320c52/artifacts', end_time=None, experiment_id='234193429068143937', lifecycle_stage='active', run_id='769e310f88e84dad8806598a23320c52', run_name='abrasive-trout-25', run_uuid='769e310f88e84dad8806598a23320c52', start_time=1688639921580, status='RUNNING', user_id='mnrozhkov'>
cnt
atemp
hum
temp
weathersit
wi

In [41]:
# from pprint import pprint

# feature = 'temp'

# # report["metrics"][1]["result"]["drift_by_columns"][feature]["drift_score"]
# pprint(list(report["metrics"][1]["result"].keys()))
# pprint(report["metrics"][1]["result"]['share_of_drifted_columns'])
# pprint(report["metrics"][1]["result"]['dataset_drift'])
# pprint(report["metrics"][0])

# data_drift_metrics = report["metrics"][0]["result"]
# pprint(data_drift_metrics)

### Option 2

In [42]:
#evaluate data drift with Evidently Profile
def detect_dataset_drift(reference, production, column_mapping, get_ratio=False):
    """
    Returns True if Data Drift is detected, else returns False.
    If get_ratio is True, returns the share of drifted features.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    Data Drift for the dataset is detected if share of the drifted features is above the selected threshold (default value is 0.5).
    """
    
    data_drift_report = Report(metrics=[
        DataDriftPreset(drift_share=0.4)
        ])
    data_drift_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)
    report = data_drift_report.as_dict()
    
    if get_ratio:
        return report["metrics"][0]["result"]["drift_share"]
    else:
        return report["metrics"][0]["result"]["dataset_drift"]
    
    
#evaluate data drift with Evidently Profile
def detect_features_drift(reference, production, column_mapping, get_scores=False):
    """
    Returns True if Data Drift is detected, else returns False. 
    If get_scores is True, returns scores value (like p-value) for each feature.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    """
    
    data_drift_report = Report(metrics=[DataDriftPreset()])
    data_drift_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)
    report = data_drift_report.as_dict()
    
    drifts = []
    num_features = column_mapping.numerical_features if column_mapping.numerical_features else []
    cat_features = column_mapping.categorical_features if column_mapping.categorical_features else []
    for feature in num_features + cat_features:
        drift_score = report["metrics"][1]["result"]["drift_by_columns"][feature]["drift_score"]
        if get_scores:
            drifts.append((feature, drift_score))
        else:
            drifts.append((feature, report["metrics"][1]["result"]["drift_by_columns"][feature]["drift_detected"]))
             
    return drifts


In [43]:
from config import MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

print(f"Client tracking uri: {client.tracking_uri}")

EXPERIMENT_NAME = "Data Drift"

# Get or Create an experiment by name 
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

if experiment: 
    
    experiment_id = experiment.experiment_id if experiment else None
    # Show experiment info
    print("Name: {}".format(experiment.name))
    print("Experiment ID: {}".format(experiment.experiment_id))
    print("Experiment Name: {}".format(experiment.name))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

else:
    
    # Create_experiment
    experiment_id = client.create_experiment(EXPERIMENT_NAME)
    print("Experiment ID: {}".format(experiment_id))

mlflow.set_experiment(EXPERIMENT_NAME)

Client tracking uri: http://localhost:5000
Name: Data Drift
Experiment ID: 234193429068143937
Experiment Name: Data Drift
Artifact Location: mlflow-artifacts:/234193429068143937
Lifecycle_stage: active


<Experiment: artifact_location='mlflow-artifacts:/234193429068143937', creation_time=1688577389525, experiment_id='234193429068143937', last_update_time=1688577389525, lifecycle_stage='active', name='Data Drift', tags={}>

In [44]:
# fig = plot_drifted_feature_scores(features_historical_drift_pvalues, experiment_batches)
# fig.show("notebook")

# fig = plot_drifted_features(features_historical_drift, experiment_batches)
# fig.show()

In [45]:
###start new run
    
with mlflow.start_run(experiment_id=experiment_id) as run: 
    
    
    features_historical_drift = []
    features_historical_drift_pvalues = []
    metrics_parent = {}

    for date in experiment_batches:
        print(date)
        
        # Calculate Dataset Drift for the fold
        with mlflow.start_run(run_name=date[1], 
                              nested=True,
                              experiment_id=experiment_id) as child_run:

            # Log parameters
            mlflow.log_param("begin", date[0])
            mlflow.log_param("end", date[1])

            # Log metrics
            report: Dict = get_data_drift_report(
                raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
                raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
                column_mapping=data_columns)
            
            metrics: Dict = get_data_drift_metrics(report.as_dict())
            ts = int(datetime.strptime(date[1], '%Y-%m-%d %H:%M:%S').timestamp())
            
            for metric in metrics.keys():
                mlflow.log_metric(
                    key=metric, 
                    value=round(metrics[metric], 3), 
                    step=ts) # use last date as step

            print(run.info)
            
            if metrics['dataset_drift'] is True:
                date2str = date[1].replace(" ", "-").replace(":", "-")
                report_path = f"data_drift_report_{date2str}.html"
                report.save_html(report_path)
                mlflow.log_artifact(report_path)
                
                metrics_parent.update({"dataset_drift": metrics['dataset_drift']})

        # Log parent Run metrics
        mlflow.log_metrics(metrics_parent)
        
        # Calculate Features Drift 
        drift_features = detect_features_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0],reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
                           column_mapping=data_columns)
        features_historical_drift.append([x[1] for x in drift_features])
        fig = plot_drifted_features(features_historical_drift, experiment_batches)
        fig.write_html("features_drift.html")
        mlflow.log_artifact("features_drift.html")
        
        
        # Calculate Features Drift scores (p-value)
        drift_feature_scores = detect_features_drift(
            raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
            raw_data.loc[raw_data.dteday.between(date[0], date[1])],
            column_mapping=data_columns,
            get_scores=True)
        features_historical_drift_pvalues.append([x[1] for x in drift_feature_scores])
        fig = plot_drifted_feature_scores(features_historical_drift_pvalues, experiment_batches)
        fig.write_html("features_drift_pvalues.html")
        mlflow.log_artifact("features_drift_pvalues.html")
    

('2011-01-01 00:00:00', '2011-01-29 23:00:00')
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
<RunInfo: artifact_uri='mlflow-artifacts:/234193429068143937/892a76fc80ce45dd8671aa7d8396310b/artifacts', end_time=None, experiment_id='234193429068143937', lifecycle_stage='active', run_id='892a76fc80ce45dd8671aa7d8396310b', run_name='wise-wren-488', run_uuid='892a76fc80ce45dd8671aa7d8396310b', start_time=1688639922783, status='RUNNING', user_id='mnrozhkov'>
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
('2011-01-29 00:00:00', '2011-02-07 23:00:00')
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
cnt
atemp
hum
temp
weathersit
windspeed
holiday
workingday
<RunInfo: artifact_uri='mlflow-artifacts:/234193429068143937/892a76fc80c

In [46]:
report.as_dict()

{'metrics': [{'metric': 'DatasetDriftMetric',
   'result': {'drift_share': 0.4,
    'number_of_columns': 8,
    'number_of_drifted_columns': 4,
    'share_of_drifted_columns': 0.5,
    'dataset_drift': True}},
  {'metric': 'DataDriftTable',
   'result': {'number_of_columns': 8,
    'number_of_drifted_columns': 4,
    'share_of_drifted_columns': 0.5,
    'dataset_drift': True,
    'drift_by_columns': {'cnt': {'column_name': 'cnt',
      'column_type': 'num',
      'stattest_name': 'K-S p_value',
      'stattest_threshold': 0.05,
      'drift_score': 0.0015453891132749989,
      'drift_detected': True,
      'current': {'small_distribution': {'x': [1107.0,
         1289.0,
         1471.0,
         1653.0,
         1835.0,
         2017.0,
         2199.0,
         2381.0,
         2563.0,
         2745.0,
         2927.0],
        'y': [0.0007849293563579278,
         0.0,
         0.0007849293563579278,
         0.0015698587127158557,
         0.0,
         0.0007849293563579278,
     

# Prdiction Monitoring

- monitor predictions for period 
- use date as a Run name 
- save report if there is a Drift 
- run from Python script

In [47]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import TargetDriftPreset, RegressionPreset

In [48]:
# Set reference dates
reference_dates = ('2011-01-01 00:00:00','2011-01-28 23:00:00')

# Set experiment batches dates
experiment_batches = [
    ('2011-01-01 00:00:00','2011-01-29 23:00:00'),
    # ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    # ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    # ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [49]:
from config import MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

EXPERIMENT_NAME = "Prediction Monitoring"

# Get or Create an experiment by name 
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

if experiment: 
    
    experiment_id = experiment.experiment_id if experiment else None
    # Show experiment info
    print("Name: {}".format(experiment.name))
    print("Experiment ID: {}".format(experiment.experiment_id))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

else:
    
    # Create_experiment
    experiment_id = client.create_experiment(
        EXPERIMENT_NAME,
        # artifact_location=MLFLOW_TRACKING_URI,
        # tags={"version": "v1", "priority": "P1"},
    )
    print("Experiment ID: {}".format(experiment_id))

Client tracking uri: http://localhost:5000
Experiment ID: 103968381533232488


In [50]:
# Evaluate data drift with Evidently Profile
# def get_data_drift_report(reference, production, column_mapping) -> Report:
#     """
#     Returns a list with pairs (feature_name, drift_score)
#     Drift Score depends on the selected statistical test or distance and the threshold
#     """    


#     return num_target_drift_report



In [51]:
columns_mapping

NameError: name 'columns_mapping' is not defined

In [ ]:
#start new run
for date in experiment_batches:
    with mlflow.start_run(experiment_id=experiment_id) as run: 
        
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Log metrics
        # report: Dict = get_data_drift_report(
        #     raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
        #     raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
        #)
        
        reference = raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])]
        production = raw_data.loc[raw_data.dteday.between(date[0], date[1])]
        column_mapping=data_columns
        
        
        num_target_drift_report = Report(metrics=[
            TargetDriftPreset(),
        ])

        num_target_drift_report.run(
            reference_data=reference, 
            current_data=production, 
            column_mapping=column_mapping)
        
        # reg_performance_report = Report(metrics=[
        #     RegressionPreset(),
        # ]) 
        # reg_performance_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)
        
        
        # metrics: Dict = get_data_drift_metrics(report.as_dict())
        
        # for metric in metrics.keys():
        #     mlflow.log_metric(metric, round(metrics[metric], 3))

        # print(run.info)

In [ ]:
target_report = num_target_drift_report.as_dict()

for metric in target_report['metrics']:
    print(metric['metric'])

ColumnDriftMetric
ColumnCorrelationsMetric
TargetByFeaturesTable
